# Testing Pipelines

<div class="alert alert-info">

This tutorial is a sequel to `02a_build_pipelines.ipynb` and will need artifacts and files stored from it. Please ensure `02a_build_pipelines.ipynb` has been run before running this tutorial.

</div>

<div class="alert alert-info">

If you encounter issues you cannot resolve, simply ask in our [Slack community](https://join.slack.com/t/lineacommunity/shared_invite/zt-18kizfn3b-1Qu_HDT3ahGudnAwoFAw9Q)'s `#support` channel. We are always happy and ready to help you!

</div>

<div class="alert alert-info">

You can ignore `# NBVAL_*` comments in certain cell blocks. They are for passing unit tests only, which we do to make sure the examples are always functional as we update the codebase.

</div>

When building a pipeline, LineaPy transforms the user's original code into modularized functions
where extraneous operations are removed. With such changes, the user may want to ensure that the
transformed code is valid and reliable before actually using it. To support this, LineaPy's pipeline
API provides an optional ``generate_test`` argument (default set to ``False``).

In [1]:
# NBVAL_IGNORE_OUTPUT
import lineapy
lineapy.tag("Pipeline Testing Demo")

As a concrete example, consider the pipeline created in `02a_build_pipelines.ipynb`, where we got the following modularized code:

In [2]:
# NBVAL_IGNORE_OUTPUT
%cat ./output/pipeline_basics/iris_pipeline_module.py

import pandas as pd
from sklearn.linear_model import LinearRegression


def get_iris_preprocessed():
    url = "https://raw.githubusercontent.com/LineaLabs/lineapy/main/examples/tutorials/data/iris.csv"
    df = pd.read_csv(url)
    color_map = {"Setosa": "green", "Versicolor": "blue", "Virginica": "red"}
    df["variety_color"] = df["variety"].map(color_map)
    df["d_versicolor"] = df["variety"].apply(lambda x: 1 if x == "Versicolor" else 0)
    df["d_virginica"] = df["variety"].apply(lambda x: 1 if x == "Virginica" else 0)
    return df


def get_iris_model(df):
    mod = LinearRegression()
    mod.fit(
        X=df[["petal.width", "d_versicolor", "d_virginica"]],
        y=df["sepal.width"],
    )
    return mod


def run_session_including_iris_preprocessed():
    # Given multiple artifacts, we need to save each right after
    # its calculation to protect from any irrelevant downstream
    # mutations (e.g., inside other artifact calculations)
    import copy

    artifacts = dict

## Generating Pipeline Test

Now, had we run:

In [3]:
# NBVAL_IGNORE_OUTPUT

# Build an Airflow pipeline using artifacts
lineapy.to_pipeline(
  pipeline_name="iris_pipeline",
  artifacts=["iris_preprocessed", "iris_model"],
  dependencies={"iris_model": {"iris_preprocessed"}},
  output_dir="./output/pipeline_basics/",
  framework="AIRFLOW",
  generate_test=True,  # Optional
)

Generated module file: output/pipeline_basics/iris_pipeline_module.py                                                                                                                                
Generated requirements file: output/pipeline_basics/iris_pipeline_requirements.txt                                                                                                                   
Generated DAG file: output/pipeline_basics/iris_pipeline_dag.py                                                                                                                                      
Generated Docker file: output/pipeline_basics/iris_pipeline_Dockerfile                                                                                                                               
Generated test scaffold file: output/pipeline_basics/test_iris_pipeline.py                                                                                                                           
Generated 

PosixPath('output/pipeline_basics')

we would have had the following file generated too for testing the modularized code:

In [4]:
# NBVAL_IGNORE_OUTPUT
%cat ./output/pipeline_basics/test_iris_pipeline.py

import os
import pickle
import unittest
import warnings
from pathlib import Path
from typing import Callable

from iris_pipeline_module import get_iris_model, get_iris_preprocessed


def safe_load_pickle(
    path_to_file: Path,
    alt_val_func: Callable = lambda: FileNotFoundError,
    save_alt_val: bool = False,
):
    """
    Load the specified pickle file if it exists.
    If not, use the provided function to generate and return
    an alternative value (the desired execution should be wrapped
    inside a lambda function to delay actual execution until needed).
    """
    if os.path.exists(path_to_file):
        with open(path_to_file, "rb") as fp:
            file_value = pickle.load(fp)
        return file_value
    else:
        alt_value = alt_val_func()
        if save_alt_val is True:
            # Store value to avoid recompute across test cases
            with open(path_to_file, "wb") as fp:
                pickle.dump(alt_value, fp)
        return alt_value


class Tes

As shown, the file contains test methods (e.g., ``TestIrisPipeline.test_get_iris_preprocessed()``) that
each examine validity of the corresponding function in the module file (e.g., ``get_iris_preprocessed()``).
Specifically, each test method attempts to check whether running the target function generates the same
output (i.e., artifact value such as pre-processed data) as the original one saved in the artifact store
(which gets copied over to a new local subfolder, e.g., ``./output/pipeline_basics/sample_output/``
in this case, during pipeline generation).

<div class="alert alert-info">

``safe_load_pickle()`` is a helper function (defined in the same testing file) that first tries to
load the pickle file at ``path_to_file`` and, if the file does not exist, uses ``alt_val_func`` to
re-calculate the value on the fly. If ``save_alt_val`` is set to ``True`` (default set to ``False``),
the value produced from ``alt_val_func`` is saved at ``path_to_file`` so that it can be used by other
related test methods without having to re-calculate it again (this "recycling" becomes critical when
the value involves long computation).

For ``alt_val_func``, the desired execution should be wrapped inside a lambda function to delay actual execution
until needed (i.e., when ``safe_load_pickle()`` realizes that the pickle file does not exist and that it needs to
re-calculate the value).

</div>

As indicated by docstrings and comments such as ``adapt as needed``, these test methods are provided as
a scaffold/template rather than a final version of pipeline testing. For instance, out of the box, all test methods
rely on ``unittest``'s ``assertEqual()`` to evaluate equality between the generated vs. expected output values
(of the function run), which may not be the right way to perform equality evaluation for certain objects such as
a NumPy array (calling ``assertEqual()`` on two NumPy arrays would return an array of multiple Boolean values
instead of a single Boolean value). Accordingly, the user is expected to customize the code to suit their own testing
needs/contexts. For example, if the tested function's output is a NumPy array, then the user can replace the templated
``self.assertEqual(sample_output_generated, sample_output_expected)`` with the customized ``assert all(sample_output_generated == sample_output_expected)``,
which would result in proper equality evaluation.

## Running Pipeline Test

With such limitations in templating, most test methods out of the box are bound to fail. However, to avoid any potential interference
with the user's system, test failures are wrapped inside the ``try``-``except`` block, which makes all test methods run successfully
without erroring out. Instead, the user is informed of failed test methods via warning messages that ask the user to check the code
and adapt it if needed.

Hence, for the ``iris_pipeline`` example discussed here, we get:

In [5]:
# NBVAL_IGNORE_OUTPUT
%cd ./output/pipeline_basics/
!python -m unittest test_iris_pipeline.TestIrisPipeline

/Users/sangyoonpark/Projects/Linea/prod/lineapy/examples/tutorials/output/pipeline_basics
/Users/sangyoonpark/Projects/Linea/prod/lineapy/examples/tutorials/output/pipeline_basics/test_iris_pipeline.py:108: UserWarning: Test failed, but this may be due to our limited templating. Please adapt the test as needed.
  warnings.warn(
./Users/sangyoonpark/Projects/Linea/prod/lineapy/examples/tutorials/output/pipeline_basics/test_iris_pipeline.py:79: UserWarning: Test failed, but this may be due to our limited templating. Please adapt the test as needed.
  warnings.warn(
.
----------------------------------------------------------------------
Ran 2 tests in 0.231s

OK


<div class="alert alert-warning">

Tests may involve long compute and/or large storage, so please take care in running them.

</div>

## Adapting Pipeline Test

As shown, both test methods "failed" (i.e., warning messages raised) for ``iris_pipeline``. This is not surprising because,
again, the scaffold is relying on a naive mode of equality evaluation via ``unittest``'s ``assertEqual()``, which does not
work for more sophisticated object types such as ``pandas.DataFrame`` and ``sklearn.linear_model.LinearRegression``.
For more proper equality evaluation, we may update `assert` statements as the following (see `try` blocks):

```python
# ./output/pipeline_basics/test_iris_pipeline.py

import os
import pickle
import unittest
import warnings
from pathlib import Path
from typing import Callable

from iris_pipeline_module import get_iris_model, get_iris_preprocessed

[...]

class TestIrisPipeline(unittest.TestCase):

    [...]

    def test_get_iris_preprocessed(self) -> None:
        """
        NOTE: The code below is provided as scaffolding/template.
        Please adapt it to your specific testing context.
        """
        # Prepare function input (adapt as needed)
        pass

        # Generate function output (adapt as needed)
        sample_output_generated = get_iris_preprocessed()

        # Perform tests (add/adapt as needed)
        sample_output_expected = safe_load_pickle(
            path_to_file=(self.art_pkl_dir / "iris_preprocessed.pkl"),
            alt_val_func=lambda: FileNotFoundError,
        )
        try:
            assert sample_output_generated.equals(sample_output_expected)
        except Exception:
            warnings.warn(
                "Test failed, but this may be due to our limited templating. "
                "Please adapt the test as needed."
            )

    def test_get_iris_model(self) -> None:
        """
        NOTE: The code below is provided as scaffolding/template.
        Please adapt it to your specific testing context.
        """
        # Prepare function input (adapt as needed)
        df = safe_load_pickle(
            path_to_file=(self.art_pkl_dir / "iris_preprocessed.pkl"),
            alt_val_func=lambda: get_iris_preprocessed(),
            save_alt_val=True,
        )

        # Generate function output (adapt as needed)
        sample_output_generated = get_iris_model(df)

        # Perform tests (add/adapt as needed)
        sample_output_expected = safe_load_pickle(
            path_to_file=(self.art_pkl_dir / "iris_model.pkl"),
            alt_val_func=lambda: FileNotFoundError,
        )
        try:
            assert all(sample_output_generated.coef_ == sample_output_expected.coef_)
        except Exception:
            warnings.warn(
                "Test failed, but this may be due to our limited templating. "
                "Please adapt the test as needed."
            )
```

With these changes, running the test would result in success without any warning messages raised.

<div class="alert alert-info">

In adapting the testing scaffold/template, we can go beyond equality evaluation. For instance, if the tested function outputs
a model, we can check whether the model behaves "reasonably" by feeding it with particular input values and observing whether
it returns output values within a certain range. This type of testing is especially valuable for models involving inherent stochasticity,
where the same procedure does not necessarily guarantee exactly identical results.

</div>